In [1]:
!pip install vosk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.2 MB 5.2 MB/s 
  Created wheel for srt: filename=srt-3.5.2-py3-none-any.whl size=22487 sha256=f55b545d1ef0700ed391e3488733b82abc42b23d3d6e8525f356ae87ad335aae
  Stored in directory: /root/.cache/pip/wheels/54/c4/ec/4604122e072aebb16803c8297b7cd3f4c72073a3ee58738015
Successfully built srt


In [2]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
from vosk import Model, KaldiRecognizer
import wave
import json
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read 

import io
import ffmpeg
import soundfile as sf
import IPython
import librosa

## Recording audio

In [4]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""


In [117]:
def get_audio():
	display(HTML(AUDIO_HTML))
	data = eval_js("data")
	binary = b64decode(data.split(',')[1])
	
	process = (ffmpeg
		.input('pipe:0')
		.output('pipe:1', format='wav')
		.run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
	)
	output, err = process.communicate(input=binary)
	
	riff_chunk_size = len(output) - 8
	# Break up the chunk size into four bytes, held in b.
	q = riff_chunk_size
	b = []
	for i in range(4):
			q, r = divmod(q, 256)
			b.append(r)

	# Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
	riff = output[:4] + bytes(b) + output[8:]
	sr, audio = read(io.BytesIO(riff))
	audio = audio/max(abs(audio))
	audio = librosa.resample(audio, sr, 16000)
	audio, _ = librosa.effects.trim(audio)
	audio = (32767*audio).astype(np.int16)
	return audio,sr

## VOSK models for Persion language

### downloading models from vosk 

In [18]:
#downloading VOSK models:
!wget -q -O /content/model_1.zip https://alphacephei.com/vosk/models/vosk-model-small-fa-0.4.zip
!wget -q -O /content/model_2.zip https://alphacephei.com/vosk/models/vosk-model-fa-0.5.zip
!wget -q -O /content/model_3.zip https://alphacephei.com/vosk/models/vosk-model-small-fa-0.5.zip

!unzip model_1.zip
!unzip model_2.zip
!unzip model_3.zip

Archive:  model_1.zip
   creating: vosk-model-small-fa-0.4/
   creating: vosk-model-small-fa-0.4/am/
  inflating: vosk-model-small-fa-0.4/am/final.mdl  
  inflating: vosk-model-small-fa-0.4/am/tree  
   creating: vosk-model-small-fa-0.4/graph/
  inflating: vosk-model-small-fa-0.4/graph/disambig_tid.int  
  inflating: vosk-model-small-fa-0.4/graph/HCLr.fst  
  inflating: vosk-model-small-fa-0.4/graph/Gr.fst  
   creating: vosk-model-small-fa-0.4/graph/phones/
 extracting: vosk-model-small-fa-0.4/graph/phones/optional_silence.csl  
 extracting: vosk-model-small-fa-0.4/graph/phones/silence.csl  
  inflating: vosk-model-small-fa-0.4/graph/phones/align_lexicon.txt  
 extracting: vosk-model-small-fa-0.4/graph/phones/optional_silence.txt  
  inflating: vosk-model-small-fa-0.4/graph/phones/word_boundary.txt  
  inflating: vosk-model-small-fa-0.4/graph/phones/disambig.int  
  inflating: vosk-model-small-fa-0.4/graph/phones/word_boundary.int  
 extracting: vosk-model-small-fa-0.4/graph/phones/op

### model and recognizer

In [145]:
audio, sr = get_audio()
sf.write('my recording.wav', audio, 16000)

In [167]:
target_txt=' سلام من نیلوفر همتی هستم من دانشجوی سال سوم رشته علوم کامپیوتر هستم و در دوره پردازش گفتار اکادمی همراه شرکت کرده ام و هدف از این صوت تبدیل گفتار به متن میباشد'

In [147]:
wf = wave.open('my recording.wav', 'rb')

In [148]:
path_1='/content/vosk-model-small-fa-0.4'

model_1=Model(lang='fa-0.5')
rec_1 = KaldiRecognizer(model_1, wf.getframerate())
rec_1.SetWords(True)

In [149]:
path_2='/content/vosk-model-fa-0.5'

model_2=Model(lang='fa-0.5')
rec_2 = KaldiRecognizer(model_2, wf.getframerate())
rec_2.SetWords(True)

In [150]:
path_3='/content/vosk-model-small-fa-0.5'

model_3=Model(lang='fa-0.5')
rec_3 = KaldiRecognizer(model_3, wf.getframerate())
rec_3.SetWords(True)

## testing the models

In [152]:
wf = wave.open('my recording.wav', 'rb')
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec_1.AcceptWaveform(data):
        print(rec_1.Result())
print(rec_1.FinalResult())

{
  "result" : [{
      "conf" : 1.000000,
      "end" : 1.140000,
      "start" : 0.390000,
      "word" : "سلام"
    }, {
      "conf" : 1.000000,
      "end" : 2.520000,
      "start" : 1.920000,
      "word" : "من"
    }, {
      "conf" : 1.000000,
      "end" : 3.360000,
      "start" : 2.640000,
      "word" : "نیلوفر"
    }, {
      "conf" : 0.925123,
      "end" : 4.110000,
      "start" : 3.360000,
      "word" : "همتی"
    }, {
      "conf" : 0.785070,
      "end" : 4.830000,
      "start" : 4.110000,
      "word" : "هستم"
    }, {
      "conf" : 1.000000,
      "end" : 5.880000,
      "start" : 5.370000,
      "word" : "من"
    }, {
      "conf" : 1.000000,
      "end" : 6.870000,
      "start" : 5.910000,
      "word" : "دانشجوی"
    }, {
      "conf" : 1.000000,
      "end" : 7.500000,
      "start" : 6.870000,
      "word" : "سال"
    }, {
      "conf" : 1.000000,
      "end" : 8.280000,
      "start" : 7.500000,
      "word" : "سوم"
    }, {
      "conf" : 1.000000,
    

In [151]:
wf = wave.open('my recording.wav', 'rb')
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec_2.AcceptWaveform(data):
        print(rec_2.Result())
print(rec_2.FinalResult())

{
  "result" : [{
      "conf" : 1.000000,
      "end" : 1.140000,
      "start" : 0.390000,
      "word" : "سلام"
    }, {
      "conf" : 1.000000,
      "end" : 2.520000,
      "start" : 1.920000,
      "word" : "من"
    }, {
      "conf" : 1.000000,
      "end" : 3.360000,
      "start" : 2.640000,
      "word" : "نیلوفر"
    }, {
      "conf" : 0.929110,
      "end" : 4.110000,
      "start" : 3.360000,
      "word" : "همتی"
    }, {
      "conf" : 0.811491,
      "end" : 4.830000,
      "start" : 4.110000,
      "word" : "هستم"
    }, {
      "conf" : 1.000000,
      "end" : 5.880000,
      "start" : 5.370000,
      "word" : "من"
    }, {
      "conf" : 1.000000,
      "end" : 6.870000,
      "start" : 5.910000,
      "word" : "دانشجوی"
    }, {
      "conf" : 1.000000,
      "end" : 7.500000,
      "start" : 6.870000,
      "word" : "سال"
    }, {
      "conf" : 1.000000,
      "end" : 8.280000,
      "start" : 7.500000,
      "word" : "سوم"
    }, {
      "conf" : 1.000000,
    

In [153]:
wf = wave.open('my recording.wav', 'rb')
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec_3.AcceptWaveform(data):
        print(rec_3.Result())
print(rec_3.FinalResult())

{
  "result" : [{
      "conf" : 1.000000,
      "end" : 1.140000,
      "start" : 0.390000,
      "word" : "سلام"
    }, {
      "conf" : 1.000000,
      "end" : 2.520000,
      "start" : 1.920000,
      "word" : "من"
    }, {
      "conf" : 1.000000,
      "end" : 3.360000,
      "start" : 2.640000,
      "word" : "نیلوفر"
    }, {
      "conf" : 0.942348,
      "end" : 4.110000,
      "start" : 3.360000,
      "word" : "همتی"
    }, {
      "conf" : 0.825644,
      "end" : 4.830000,
      "start" : 4.110000,
      "word" : "هستم"
    }, {
      "conf" : 1.000000,
      "end" : 5.880000,
      "start" : 5.370000,
      "word" : "من"
    }, {
      "conf" : 1.000000,
      "end" : 6.900000,
      "start" : 5.910000,
      "word" : "دانشجوی"
    }, {
      "conf" : 1.000000,
      "end" : 7.500000,
      "start" : 6.900000,
      "word" : "سال"
    }, {
      "conf" : 1.000000,
      "end" : 8.280000,
      "start" : 7.500000,
      "word" : "سوم"
    }, {
      "conf" : 1.000000,
    

In [163]:
result = "سلام من نیلوفر همتی هستم من دانشجوی سال سوم رشته علوم کامپیوتر هستند و در دوره پردازش گفتار آکادمی همراه شرکت کرد و هدف از این سرعت تبدیل گفتگوها در نت می‌باشند"

## resualt

In [169]:
print("target  : ",target_txt)
print("resualt : ",result)

target  :   سلام من نیلوفر همتی هستم من دانشجوی سال سوم رشته علوم کامپیوتر هستم و در دوره پردازش گفتار اکادمی همراه شرکت کرده ام و هدف از این صوت تبدیل گفتار به متن میباشد
resualt :  سلام من نیلوفر همتی هستم من دانشجوی سال سوم رشته علوم کامپیوتر هستند و در دوره پردازش گفتار آکادمی همراه شرکت کرد و هدف از این سرعت تبدیل گفتگوها در نت می‌باشند


In [172]:
dels=1 #به
ins=1 #در 
subs=5 #هستند ، کرده‌ام ، سرعت ، گفتگوها ، نت
WER = 100*(subs+ins+dels)/32

print("WER of three models : ",WER)

WER of three models :  21.875
